In [1]:
from pathlib import Path
from indigo import Indigo
import numpy as np
import pandas as pd

from substrucure_finder import BucketsInitializer
from substrucure_finder import SearchEngine

import fp_utils
from fp_utils.finders import Finder
from fp_utils.tests import FinderSpeedTester
from fp_utils.catch_time import CatchTime

In [2]:
fp_utils.settings.init_fp_utils()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
data_path = Path("../data/search_data_2/")

In [4]:
zero_columns_name = '../data/zero_columns'

# Build structure

In [5]:
initializer = BucketsInitializer(data_path, columns_count=100)

In [6]:
from substrucure_finder import consts

In [7]:
consts.fingerprint_size_in_bits

2584

In [8]:
# file = data_path / "raw_buckets/55/0.csv"

In [9]:
# with file.open('r') as f:
#     lines = f.readlines()
#     line = lines[105]

In [10]:
# line

In [11]:
# df = pd.read_csv(file, delimiter=',', header=0, index_col=0, dtype=dict((str(i), bool) for i in range(3)))

In [12]:
# df

In [13]:
(data_path / 'buckets').mkdir(parents=True, exist_ok=True)

In [ ]:
%%time
initializer.init_buckets()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Start init 61
Finish init 61
Start init 62
Finish init 62
Start init 77
Finish init 77
Start init 78
Finish init 78
Start init 79
Finish init 79
Start init 80
Finish init 80
Start init 81
Finish init 81
Start init 82
Finish init 82
Start init 83
Finish init 83
Start init 84
Finish init 84
Start init 85
Finish init 85
Start init 86
Finish init 86
Start init 87
Finish init 87
Start init 88
Finish init 88
Start init 89
Finish init 89
Start init 90
Finish init 90
Start init 91
Finish init 91
Start init 92
Finish init 92
Start init 94
Finish init 94
Start init 95
Finish init 95
Start init 97
Finish init 97
Start init 98
Finish init 98
Start init 99
Finish init 99
Start init 100
Finish init 100
Start init 111
Finish init 111
Start init 112
Finish init 112
Start init 117
Finish init 117
Start init 118
Finish init 118
Start init 119
Finish init 119
Start

# Search Queries

In [ ]:
with open(zero_columns_name, 'r') as f:
    zero_columns = list(map(int, f.read().split()))
assert list(sorted(zero_columns)) == zero_columns

In [ ]:
full_columns = list(sorted(set(range(3736)) - set(zero_columns)))

In [ ]:
def bin_format(number, num_of_bits=3736):
    fp = list(map(int, list(bin(int(number, 16))[2:].zfill(num_of_bits))))
    return np.fromiter(fp, dtype=int)

In [ ]:
def smiles_to_fingerprint(smiles):
    indigo_mol_to_test = Indigo().loadMolecule(smiles)
    full_fp = bin_format(indigo_mol_to_test.fingerprint("sub").toString())
    return np.fromiter(pd.Series(full_fp)[full_columns].values, dtype=int)

In [ ]:
@CatchTime("search")
def search(smiles: str, finder: Finder, ans_count=None):
    fp = smiles_to_fingerprint(smiles)
    return list(finder.find(fp, ans_count))

In [ ]:
class ComplexFinder(SearchEngine, Finder):   
    def find_all(self, fp):
        return self.search(fp)

In [ ]:
complex_finder = ComplexFinder(data_path)

In [ ]:
with open('../data/pubchem_994_queries.txt', 'r') as f:
    queries = list(map(lambda x: x.split()[0], f.read().strip().split('\n')))

In [ ]:
len(queries), queries[:10]

In [ ]:
for q in queries:
    print(f'\nQuery: {q}')
    answers = list(search(q, complex_finder, ans_count=10))
    print(f'Ans len: {len(answers)}')